In [ ]:
from models.cnn import CNN
import time
import gc

import pandas as pd
import csv
import base64
import io
from PIL import Image
import torch
from tabulate import tabulate
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
idimage_df = pd.read_csv("data/idimage_fixed.csv")
idlabel_df = pd.read_csv("data/idlabel.csv")
idmeta_df  = pd.read_csv("data/idmeta.csv")

In [ ]:
idlabel_df['isfraud'] = idlabel_df['isfraud'].astype(bool)

In [ ]:
def predict_fraud(image_b64):
    try:
        img_bytes = base64.b64decode(image_b64)
        img = Image.open(io.BytesIO(img_bytes)).convert("RGB").resize((128, 128))
        arr = np.array(img) / 255.0
        arr = np.expand_dims(arr, 0).transpose((0, 3, 1, 2))
        tensor = torch.from_numpy(arr).float()
        with torch.no_grad():
            out = model(tensor)
        return bool(out[0][0] > 0.5)
    except Exception:
        return False


In [ ]:

def fraud_predicted(image_df):
    image_df = image_df.copy()
    image_df['predicted_fraud'] = image_df['imageData'].apply(predict_fraud)
    total_ids = len(image_df)
    fraud_predicted = image_df['predicted_fraud'].sum()
    fraud_rate_percentage = 100.0 * fraud_predicted / total_ids
    return {
        'total_ids': total_ids,
        'fraud_predicted': fraud_predicted,
        'fraud_rate_percentage': fraud_rate_percentage
    }

In [ ]:

def fraud_ground_truth(label_df):
    total_ids = len(label_df)
    total_fraud = label_df['isfraud'].sum()
    total_nonfraud = total_ids - total_fraud
    fraud_percentage = 100.0 * total_fraud / total_ids
    return {
        'total_ids': total_ids,
        'total_fraud': total_fraud,
        'total_nonfraud': total_nonfraud,
        'fraud_percentage': fraud_percentage
    }

In [ ]:

def fraud_pattern_ethnicity_wise(image_df, label_df, meta_df):
    image_df = image_df.copy()
    image_df['predicted_fraud'] = image_df['imageData'].apply(predict_fraud)
    merged_df = image_df.merge(label_df, left_on='name', right_on='id')
    merged_df = merged_df.merge(meta_df, on='id')
    group = merged_df.groupby('ethnicity').agg(
        total_customers=('id', 'count'),
        predicted_fraud=('predicted_fraud', 'sum')
    ).reset_index()
    group['fraud_rate_pct'] = 100.0 * group['predicted_fraud'] / group['total_customers']
    return group.sort_values('fraud_rate_pct', ascending=False).head(10)

In [ ]:

def fraud_rate_by_veteran_status(image_df, label_df, meta_df):
    image_df = image_df.copy()
    image_df['predicted_fraud'] = image_df['imageData'].apply(predict_fraud)
    merged_df = image_df.merge(label_df, left_on='name', right_on='id')
    merged_df = merged_df.merge(meta_df, on='id')
    group = merged_df.groupby('is_veteran').agg(
        total_individuals=('id', 'count'),
        predicted_fraud=('predicted_fraud', 'sum')
    ).reset_index()
    group['fraud_rate_pct'] = 100.0 * group['predicted_fraud'] / group['total_individuals']
    return group.sort_values('fraud_rate_pct', ascending=False).head(10)

In [ ]:
timings = {
    'fraud_predicted': {'spark': [], 'pandas': []},
    'fraud_ground_truth': {'spark': [], 'pandas': []},
    'fraud_pattern_ethnicity_wise': {'spark': [], 'pandas': []},
    'fraud_rate_by_veteran_status': {'spark': [], 'pandas': []}
}


In [ ]:

for scale_name, factor in scales:
    # print(f"=== {scale_name} Done===")
    
    img_df_scaled = pd.concat([idimage_df] * factor, ignore_index=True)
    lbl_df_scaled = pd.concat([idlabel_df] * factor, ignore_index=True)
    meta_df_scaled = pd.concat([idmeta_df] * factor, ignore_index=True)
    
    # Measure Pandas execution times
    start = time.time()
    fraud_predicted(img_df_scaled)
    timings['fraud_predicted']['pandas'].append(time.time() - start)
    
    start = time.time()
    fraud_ground_truth(lbl_df_scaled)
    timings['fraud_ground_truth']['pandas'].append(time.time() - start)
    
    start = time.time()
    fraud_pattern_ethnicity_wise(img_df_scaled, lbl_df_scaled, meta_df_scaled)
    timings['fraud_pattern_ethnicity_wise']['pandas'].append(time.time() - start)
    
    start = time.time()
    fraud_rate_by_veteran_status(img_df_scaled, lbl_df_scaled, meta_df_scaled)
    timings['fraud_rate_by_veteran_status']['pandas'].append(time.time() - start)
    gc.collect()
    print(f"=== {scale_name} Done===")

In [ ]:
xticks = [0, 20, 40, 60, 80, 100]
print(x_ticks)
scale_values = [1, 5, 10, 20, 50, 100]
print(scale_values)

In [ ]:
for query_name in timings:
    plt.figure(figsize=(10, 6))
    plt.plot(scale_values, timings[query_name]['pandas'], marker='s', label='Pandas')
    plt.xlabel('Data Scale')
    plt.ylabel('Execution Time (s)')
    plt.title(f'Execution Time vs Data Scale for {query_name}')
    plt.legend()
    plt.grid(True)
    plt.xticks(xticks)
    plt.tight_layout()
    plt.savefig(f"plots/pandas/{query_name}_pandas.png")
    plt.close()


In [ ]:
print("\n=== Pandas Query Execution Times (seconds) ===")

header = ["Query"] + scale_labels
rows = [
    [qname] + [f"{t:.2f}" for t in timings[qname]["pandas"]]
    for qname in timings
]

print(tabulate(rows, headers=header, tablefmt="github"))